# Urea with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up.

In [1]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict,
                    create_construction_instructions, refine, write_cif,
                    write_res, write_fcf, ConstrainedValues,
                    create_atom_table)
import os

In [2]:
from xharpy.core import construct_esds

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
filename = 'xharpy'

In [5]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [6]:
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [7]:
hkl = shelxl_hkl2pd('iam.hkl')

In [8]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

In [9]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
}

In [10]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [11]:
refinement_dict['core_io'] = ('save', os.path.join(output_folder, 'f0j_core.pic'))

computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.225,
    'gridinterpolation': 4,
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-04 18:46:16.098601
Preparing
  calculating the core structure factor for O
  calculating the core structure factor for N
  calculating the core structure factor for H
  calculating the core structure factor for C
  saved core atomic form factors from disk
  calculating first atomic form factors
  calculated density with energy -104.69534056074443
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.057945, nit: 1, Optimization terminated successfully.
  minimizing least squares sum
  wR2: 0.021281, nit: 44, Optimization terminated successfully.
step 1: calculating new structure factors
  calculated density with energy -108.16907155667268
  minimizing least squares sum
  wR2: 0.017794, nit: 37, Optimization terminated successfully.
step 2: calculating new structure factors
  calculated density with energy -108.16037339448297
  minimizing least squares sum
  wR2: 0.017928, nit: 33, Optimization terminated successfully

In [12]:
refinement_dict['core_io'] = ('load', os.path.join(output_folder, 'f0j_core.pic'))

computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.125,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (2, 2, 2), 'gamma': True},
    #'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-04 18:50:18.362221
Preparing
  loaded core atomic form factors from disk
  calculating first atomic form factors
  calculated density with energy -107.51867024835322
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.018111, nit: 1, Optimization terminated successfully.
  minimizing least squares sum
  wR2: 0.017494, nit: 32, Optimization terminated successfully.
step 1: calculating new structure factors
  calculated density with energy -107.52331413754592
  minimizing least squares sum
  wR2: 0.017478, nit: 31, Optimization terminated successfully.
step 2: calculating new structure factors
  calculated density with energy -107.52285136030288
  minimizing least squares sum
  wR2: 0.017479, nit: 30, Optimization terminated successfully.
step 3: calculating new structure factors
  calculated density with energy -107.52290521346778
  minimizing least squares sum
  wR2: 0.017479, nit: 31, Optimization terminated succe

In [13]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [14]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [15]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [16]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [17]:
atom_table = create_atom_table(cell, construction_instructions, parameters, var_cov_mat)

In [18]:
constraint_dict['O']['cijk'] = ConstrainedValues(
    variable_indexes = (-1, -1, 0, -1, -1, 1, -1, 1, -1, 2),
    multiplicators = (0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0),
    added_value = tuple([0]* 10),
    special_position=True
)
constraint_dict['N']['cijk'] = ConstrainedValues(
    variable_indexes=(0, 0, 1, 2, 2, 3, 4, 3, 4, 5),
    multiplicators=tuple([1.0]*10),
    added_value=tuple([0.0]*10),
    special_position=True
)


construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict,
    atoms_for_gc3=['N', 'O'],
    scaling0=float(parameters[0])
)

In [19]:
refinement_dict['reload_step'] = 0

In [20]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-04 19:08:18.947493
Preparing
  loaded core atomic form factors from disk
  calculating first atomic form factors
  calculated density with energy -107.52289978473223
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.017386, nit: 1, Optimization terminated successfully.
  minimizing least squares sum
  wR2: 0.013981, nit: 46, Desired error not necessarily achieved due to precision loss.
step 1: calculating new structure factors
  calculated density with energy -107.52010270597374
  minimizing least squares sum
  wR2: 0.013993, nit: 46, Desired error not necessarily achieved due to precision loss.
step 2: calculating new structure factors
  calculated density with energy -107.52063337098409
  minimizing least squares sum
  wR2: 0.013992, nit: 44, Desired error not necessarily achieved due to precision loss.
step 3: calculating new structure factors
  calculated density with energy -107.52058071536527
  minimizing l

In [21]:
atom_table = create_atom_table(cell, construction_instructions, parameters, var_cov_mat)

constraint_dict['O']['dijkl'] = ConstrainedValues(
    variable_indexes=(0, 0, 1, 2, 2, -1, -1, -1, -1, 3, 4, 4, -1, -1, 5),
    multiplicators=(1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0),
    added_value=tuple([0.0]*15)
)

constraint_dict['N']['dijkl'] = ConstrainedValues(
    variable_indexes=(0, 0, 1, 2, 2, 3, 4, 3, 4, 5, 6, 6, 7, 7, 8),
    multiplicators=tuple([1.0]*15),
    added_value=tuple([0.0]*15)
)
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict,
    atoms_for_gc3=['N', 'O'],
    atoms_for_gc4=['N', 'O'],
    scaling0=float(parameters[0])
)

In [22]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-04 19:21:16.345936
Preparing
  loaded core atomic form factors from disk
  calculating first atomic form factors
  calculated density with energy -107.5205860815965
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.013992, nit: 1, Optimization terminated successfully.
  minimizing least squares sum
  wR2: 0.013212, nit: 53, Desired error not necessarily achieved due to precision loss.
step 1: calculating new structure factors
  calculated density with energy -107.5279052818479
  minimizing least squares sum
  wR2: 0.013185, nit: 59, Desired error not necessarily achieved due to precision loss.
step 2: calculating new structure factors
  calculated density with energy -107.52757341538435
  minimizing least squares sum
  wR2: 0.013187, nit: 54, Desired error not necessarily achieved due to precision loss.
step 3: calculating new structure factors
  calculated density with energy -107.52760495393846
  minimizing lea

In [23]:
filename = 'xharpy_dijkl'

In [24]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [25]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [26]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [27]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);